In [1]:
import pandas as pd
from datetime import datetime, timedelta


In [2]:

import pandas as pd
from datetime import datetime, timedelta

beh_columns = ['ID', 'UserID', 'Time', 'History', 'Impression']
beh_df = pd.read_csv('behaviors.tsv', sep='\t', names=beh_columns, usecols=[0, 1, 2, 3, 4])

news_columns = ['ID', 'Category', 'Subcategory', 'Title', 'Abstract']
news_df = pd.read_csv('news.tsv', sep='\t', names=news_columns, usecols=[0, 1, 2, 3, 4])


In [18]:
from most_popular_recommender import most_popular_recommender
from tf_idf_recommender import tf_idf_cosine_recommender
import numpy as np

from similiar_category_combinations import sim_cat_rec
from random_recommender import random_rating_recommender

def get_ids_old_24(beh_df, date):
	
	date_ = datetime.strptime(date, '%m/%d/%Y %I:%M:%S %p')
	start_date_ = date_ - timedelta(hours=1)

	beh_df_copy = beh_df.copy()

	# Convert the 'Time' column to datetime objects
	beh_df_copy['Time'] = pd.to_datetime(beh_df_copy['Time'], format='%m/%d/%Y %I:%M:%S %p')

	# Filter the DataFrame based on the 'Time' column
	filtered_df = beh_df_copy[(beh_df_copy['Time'] >= start_date_) & (beh_df_copy['Time'] <= date)]
	split_ids = filtered_df['Impression'].str.split()
	
	all_article_ids = []

	for ids in split_ids:
		for id in ids:
			if id.endswith('1'):
				all_article_ids.append(id)

	return all_article_ids

def best_score_ids(ids, scores_array):

	df = pd.DataFrame({'scores': scores_array, 'id': range(len(scores_array))})

	df['ImpressionID'] = [(ids[i]).split("-")[0] for i in df['id']]
	df = df.drop_duplicates(subset='ImpressionID')
	top_5_impression_ids = df.sort_values(by='scores', ascending=False).head(5)['ImpressionID'].values

	return top_5_impression_ids

def get_reccomendations():
	row_id = 46

	date = beh_df[beh_df['ID'] == row_id]['Time'].values[0]
	row_unique = beh_df.iloc[[row_id]].copy()
	row_unique.reset_index(drop=True, inplace=True)


	row_notunique = beh_df.iloc[[row_id]].copy()
	row_notunique.reset_index(drop=True, inplace=True)

	ids = get_ids_old_24(beh_df, date)
	unique_ids = list(set(ids))

	row_unique.at[0, 'Impression'] = ' '.join(unique_ids)
	row_notunique.at[0, 'Impression'] = ' '.join(ids)

	most_popular = most_popular_recommender(row_notunique,news_df=[])
	scores = most_popular['Scores'].values[0]

	print(best_score_ids(ids, scores))

	# print(tf_idf_cosine_recommender(row_unique, news_df))
	
	# userid = beh_df[beh_df['ID'] == row_id]['UserID'].values[0]
	# print(sim_cat_rec(row_notunique, news_df, userid))
	
	# random_rating_recommender()
	

get_reccomendations()

	

['N287-1' 'N59673-1' 'N51048-1' 'N47061-1' 'N34876-1']
